In [1]:
#on va scrap google maps
#link : https://www.google.fr/maps/search/restaurant+paris/@48.8701745,2.2726075,18356m/data=!3m2!1e3!4b1?hl=fr&entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D
link="https://www.google.fr/maps/search/restaurant+paris/@48.8701745,2.2726075,18356m/data=!3m2!1e3!4b1?hl=fr&entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"

In [2]:
#xpath
#nom resto
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[1]/h1

#note
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]

#prix
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/span/span/span/span[2]/span/span

#adresse
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[11]/div[3]/button/div/div[2]/div[1]

#description
# /html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[8]/button/div[1]


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
import time
import random
import csv

In [4]:
# Configuration du WebDriver avec des options
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--headless")

# User-Agent Rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
]
options.add_argument(f"user-agent={random.choice(user_agents)}")


In [5]:
def scrape_google_maps(link, n, output_file):
    """
    Fonction pour scraper les informations des restaurants à partir de Google Maps.

    :param link: URL de la recherche Google Maps
    :param n: Nombre de restaurants à scraper
    :param output_file: Nom du fichier CSV pour sauvegarder les résultats
    """
    # Initialisation du WebDriver
    driver = webdriver.Firefox(options=options)
    driver.get(link)

    # Attente pour charger la page et accepter les cookies
    time.sleep(5)
    try:
        cookies_button = driver.find_element(By.CSS_SELECTOR, ".VtwTSb > form:nth-child(2) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)")
        cookies_button.click()
        time.sleep(2)
    except Exception as e:
        print("Pas de bouton cookies trouvé :", e)

    # Définir l'élément de scroll avec le XPath
    scroll_xpath = '//div[@class="XltNde tTVLSc"]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]'
    try:
        scrollable_div = driver.find_element(By.XPATH, scroll_xpath)
        print("Élément de scroll trouvé avec succès !")
    except Exception as e:
        print(f"Erreur lors de la recherche de l'élément de scroll : {e}")
        driver.quit()
        return

    # Fonction pour récupérer les informations d'un restaurant
    def get_restaurant_info():
        try:
            name = driver.find_element(By.CSS_SELECTOR, ".DUwDvf").text
        except:
            name = "NaN"
        try:
            rating = driver.find_element(By.CSS_SELECTOR, ".F7nice > span:nth-child(1) > span:nth-child(1)").text
        except:
            rating = "NaN"
        try:
            price = driver.find_element(By.CSS_SELECTOR, ".mgr77e > span:nth-child(1) > span:nth-child(2) > span:nth-child(1) > span:nth-child(1)").text
        except:
            price = "NaN"
        try:
            restaurant_type = driver.find_element(By.CSS_SELECTOR, "button.DkEaL").text
        except:
            restaurant_type = "NaN"
        try:
            description = driver.find_element(By.CSS_SELECTOR, ".PYvSYb").text
        except:
            description = "NaN"
        try:
            address = driver.find_element(By.CSS_SELECTOR, "div.RcCsl:nth-child(3) > button:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text
        except:
            address = "NaN"
        return {
            "name": name,
            "rating": rating,
            "price": price,
            "restaurant_type": restaurant_type,
            "description": description,
            "address": address
        }

    # Collecte des données des restaurants
    restaurants_data = []
    restaurant_index = 1

    while restaurant_index <= n:
        try:
            # XPath dynamique pour sélectionner chaque restaurant
            restaurant_xpath = f"/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{(restaurant_index+1)*2-1}]/div/a"
            
            # Scrolling jusqu'à ce que l'élément soit trouvé
            max_scroll_attempts = 25 # Limite de scrolls pour éviter une boucle infinie
            scroll_attempts = 0
            while scroll_attempts < max_scroll_attempts:
                #si on est au 20eme scroll on attend 10sec
                if scroll_attempts == 24:
                    time.sleep(15)
                try:
                    restaurant = driver.find_element(By.XPATH, restaurant_xpath)
                    break  # Si l'élément est trouvé, on sort de la boucle
                except:
                    # Scroll vers le bas pour charger plus d'éléments
                    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
                    time.sleep(random.uniform(1, 2))  # Pause pour charger les nouveaux éléments
                    scroll_attempts += 1

            # Si l'élément n'est toujours pas trouvé après le scroll, arrêter
            if scroll_attempts >= max_scroll_attempts:
                print(f"Impossible de trouver le restaurant {restaurant_index} après plusieurs tentatives de scroll.")
                break

            # Cliquer sur le restaurant trouvé
            restaurant.click()
            time.sleep(3)  # Temps pour charger les informations

            # Récupération des informations
            info = get_restaurant_info()
            restaurants_data.append(info)
            print(f"Restaurant {restaurant_index} : {info}")

            # Retour à la liste
            driver.back()
            time.sleep(random.uniform(2, 4))  # Pause avant de continuer

            restaurant_index += 1
        except Exception as e:
            print(f"Erreur avec le restaurant {restaurant_index} : {e}")
            break

    # Sauvegarde des données dans un fichier CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["name", "rating", "price", "restaurant_type", "description", "address"])
        writer.writeheader()
        writer.writerows(restaurants_data)

    print(f"Données sauvegardées dans {output_file}")
    # Fermeture du navigateur
    driver.quit()


In [6]:
n=100

In [7]:
#pour paris 1er arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+1er/@48.8631336,2.3262864,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_1er.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Au Pied de Cochon', 'rating': '4,3', 'price': '€€€', 'restaurant_type': 'Restaurant français', 'description': 'Brasserie simple proposant petit-déjeuner, plats locaux et plateaux de charcuterie, accompagnés de bières pression et de cocktails.', 'address': '6 Rue Coquillière, 75001 Paris'}
Restaurant 2 : {'name': 'Palais Royal Restaurant', 'rating': '4,6', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant gastronomique', 'description': 'Vins et cuisine française gastronomique dans une salle à manger raffinée ou en terrasse dans les jardins du palais.', 'address': '110 Gal de Valois, 75001 Paris'}
Restaurant 3 : {'name': 'Palais Royal Restaurant', 'rating': '4,6', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant gastronomique', 'description': 'Vins et cuisine française gastronomique dans une salle à manger raffinée ou en terrasse dans les jardins du palais.', 'address': '110 Gal de Valois, 75001 Paris'}
Restaurant 4

In [8]:
#pour paris 2eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+2eme/@48.867583,2.3312493,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_2eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Restaurant Clémentine - Bistrot Terrasse Paris 2', 'rating': '4,7', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': 'Plats de brasserie français traditionnels et vins régionaux dans un restaurant cosy avec terrasse côté rue.', 'address': '5 Rue Saint-Marc, 75002 Paris'}
Restaurant 2 : {'name': 'Brasserie Rosie Lou', 'rating': '4,7', 'price': '20–30 €', 'restaurant_type': 'Brasserie', 'description': 'NaN', 'address': '31 Bd de Bonne Nouvelle, 75002 Paris'}
Restaurant 3 : {'name': "Le Nom M'échappe", 'rating': '4,5', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': "Steaks grillés, fruits de mer et plats de brasserie raffinés dans un restaurant clair traditionnel doté d'une terrasse côté rue.", 'address': '28 Rue Notre Dame des Victoires, 75002 Paris'}
Restaurant 4 : {'name': 'La Petite Régalade', 'rating': '4,4', 'price': '30–50 €', 'restaurant_type': 'Restaurant français', 'de

In [9]:
#pour paris 3eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+3/@48.8675824,2.3312493,1993m/data=!3m1!1e3?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_3eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Yansai 3 - Restaurant asiatique Paris - Guabao Taiwanais', 'rating': '4,8', 'price': '10–20 €', 'restaurant_type': 'Restaurant asiatique', 'description': 'NaN', 'address': '88 Rue des Gravilliers, 75003 Paris'}
Restaurant 2 : {'name': 'Yansai 3 - Restaurant asiatique Paris - Guabao Taiwanais', 'rating': '4,8', 'price': '10–20 €', 'restaurant_type': 'Restaurant asiatique', 'description': 'NaN', 'address': '88 Rue des Gravilliers, 75003 Paris'}
Restaurant 3 : {'name': 'Au Fil des Saisons', 'rating': '4,4', 'price': '30–40 €', 'restaurant_type': 'Restaurant français', 'description': "Restaurant chaleureux doté de murs en pierres apparentes et d'un patio, et proposant des plats traditionnels à base de produits de saison.", 'address': '6 Rue des Fontaines du Temple, 75003 Paris'}
Restaurant 4 : {'name': 'Robert et Louise', 'rating': '4,5', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': 'Steak, confit de canard e

In [10]:
#pour paris 4eme arrondissement:   
link="https://www.google.fr/maps/search/restaurant+paris+4/@48.8632801,2.3384501,3987m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_4eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Au Bourguignon du Marais', 'rating': '4,6', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': 'Escargots, bœuf bourguignon et autres plats traditionnels de Bourgogne servis dans un restaurant élégant.', 'address': '52 Rue François Miron, 75004 Paris'}
Restaurant 2 : {'name': 'Le Colimaçon Marais', 'rating': '4,6', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': 'Version créative des plats de brasserie française dans un cadre chaleureux avec murs en pierre brute et poutres en bois.', 'address': '44 Rue Vieille du Temple, 75004 Paris'}
Restaurant 3 : {'name': 'Salento Marais', 'rating': '4,6', 'price': '20–30 €', 'restaurant_type': 'Restaurant italien', 'description': 'Antipasti, pâtes et plats italiens à la présentation soignée servis dans un restaurant élégant de style rétro chic avec terrasse.', 'address': '6 Rue de Jarente, 75004 Paris'}
Restaurant 4 : {'name': 'Le Trésor', '

In [11]:
#pour paris 5eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+5/@48.8633329,2.3384501,3987m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_5eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Bistrot La Grange - Restaurant Paris 5', 'rating': '4,6', 'price': '10–20 €', 'restaurant_type': 'Restaurant', 'description': 'Ce bistrot propose une cuisine traditionnelle aux accents méditerranéens dans un cadre typique avec terrasse.', 'address': '17 Rue Lagrange, 75005 Paris'}
Restaurant 2 : {'name': 'La Table de Colette', 'rating': '4,7', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant gastronomique', 'description': "Restaurant écoresponsable et gastronomique doté d'une terrasse, proposant des menus de 3 à 7 plats avec des accords mets et vins.", 'address': '17 Rue Laplace, 75005 Paris'}
Restaurant 3 : {'name': 'La Truffière', 'rating': '4,5', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant français', 'description': 'Restaurant pittoresque avec poutres en bois et voûtes en pierre, servant une cuisine française raffinée à la présentation soignée.', 'address': '4 Rue Blainville, 75005 Paris'}
Restaurant 4 : 

In [12]:
#pour paris 6eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+6/@48.8676725,2.3003494,7973m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_6eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Paris Paris', 'rating': '4,4', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': 'NaN', 'address': '8 Rue de Montfaucon, 75006 Paris'}
Restaurant 2 : {'name': 'Le Paris Paris', 'rating': '4,4', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': 'NaN', 'address': '8 Rue de Montfaucon, 75006 Paris'}
Restaurant 3 : {'name': 'Brasserie Des Prés', 'rating': '4,6', 'price': '20–40 €', 'restaurant_type': 'Restaurant français', 'description': 'NaN', 'address': '6 Cr du Commerce Saint-André, 75006 Paris'}
Restaurant 4 : {'name': 'Didon', 'rating': '4,7', 'price': 'NaN', 'restaurant_type': 'Restaurant gastronomique', 'description': "Restaurant à l'ambiance décontractée et au cadre chic et simple, spécialisé dans les grillades au charbon de bois.", 'address': '8 Rue du Dragon, 75006 Paris'}
Restaurant 5 : {'name': "L'Atelier Roulière", 'rating': '4,6', 'price': '€€', 'restaurant_type': 'Restaurant', 'descr

In [13]:
#pour paris 7eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+6/@48.8676725,2.3003494,7973m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_7eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Paris Paris', 'rating': '4,4', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': 'NaN', 'address': '8 Rue de Montfaucon, 75006 Paris'}
Restaurant 2 : {'name': 'Le Paris Paris', 'rating': '4,4', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': 'NaN', 'address': '8 Rue de Montfaucon, 75006 Paris'}
Restaurant 3 : {'name': 'Brasserie Des Prés', 'rating': '4,6', 'price': '20–40 €', 'restaurant_type': 'Restaurant français', 'description': 'NaN', 'address': '6 Cr du Commerce Saint-André, 75006 Paris'}
Restaurant 4 : {'name': 'Didon', 'rating': '4,7', 'price': 'NaN', 'restaurant_type': 'Restaurant gastronomique', 'description': "Restaurant à l'ambiance décontractée et au cadre chic et simple, spécialisé dans les grillades au charbon de bois.", 'address': '8 Rue du Dragon, 75006 Paris'}
Restaurant 5 : {'name': "L'Atelier Roulière", 'rating': '4,6', 'price': '€€', 'restaurant_type': 'Restaurant', 'descr

In [14]:
#pour paris 8eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+8/@48.8744256,2.2970739,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_8eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Da Alfredo Restaurant Paris 8', 'rating': '4,7', 'price': 'NaN', 'restaurant_type': 'Restaurant italien', 'description': 'NaN', 'address': '182 Bd Haussmann, 75008 Paris'}
Restaurant 2 : {'name': 'Restaurant Super Huit Paris 8', 'rating': '4,8', 'price': '50–60 €', 'restaurant_type': 'Restaurant', 'description': 'NaN', 'address': '95 Rue de Miromesnil, 75008 Paris'}
Restaurant 3 : {'name': 'Sphère', 'rating': '4,8', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant gastronomique', 'description': 'NaN', 'address': '18 Rue La Boétie, 75008 Paris'}
Restaurant 4 : {'name': 'Le Marignan', 'rating': '4,2', 'price': 'NaN', 'restaurant_type': 'Restaurant', 'description': 'Cette brasserie conviviale aux grandes baies vitrées propose des plats traditionnels en service non-stop.', 'address': '18 Rue de Marignan, 75008 Paris'}
Restaurant 5 : {'name': 'ONOR par Thierry Marx', 'rating': '4,8', 'price': '+ de 100 €', 'restaurant_type

In [15]:
#pour paris 9eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+9/@48.8744884,2.266174,7972m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_9eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Mieux', 'rating': '4,7', 'price': 'NaN', 'restaurant_type': 'Restaurant', 'description': "Restaurant rustique-chic à l'ambiance détendue et à l'approvisionnement responsable, proposant des plats et des vins.", 'address': '21 Rue Saint-Lazare, 75009 Paris'}
Restaurant 2 : {'name': 'Les Saisons', 'rating': '4,7', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': 'Restaurant cozy et lumineux proposant des plats classiques, de la viande et une vaste sélection de vins.', 'address': '52 Rue Lamartine, 75009 Paris'}
Restaurant 3 : {'name': 'Poppy Paris', 'rating': '4,6', 'price': '20–30 €', 'restaurant_type': 'Restaurant brunch', 'description': 'NaN', 'address': '2 Rue Milton, 75009 Paris'}
Restaurant 4 : {'name': 'La Padellina', 'rating': '4,8', 'price': '20–30 €', 'restaurant_type': 'Restaurant italien', 'description': 'NaN', 'address': '1 Rue Chaptal, 75009 Paris'}
Restaurant 5 : {'name': 'Passionné', 'ra

In [16]:
#pour paris 10eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+10/@48.8756274,2.3478646,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_10eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Les Rupins - Restaurant Parisien', 'rating': '4,6', 'price': '30–40 €', 'restaurant_type': 'Restaurant', 'description': 'Restaurant simple servant des plats européens classiques, y compris des burgers, des tartares, du gaspacho et des lasagnes.', 'address': '35 Bd de Magenta, 75010 Paris'}
Restaurant 2 : {'name': 'LArchimede - Restaurant Bistronomique Paris 10', 'rating': '4,8', 'price': '30–50 €', 'restaurant_type': 'Restaurant français', 'description': 'NaN', 'address': '11 Rue Marie et Louise, 75010 Paris'}
Restaurant 3 : {'name': 'Restaurant Mâche', 'rating': '4,8', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant gastronomique', 'description': 'Menus dégustation innovants et vins naturels proposés dans un espace chic à la décoration originale.', 'address': '61 Rue de Chabrol, 75010 Paris'}
Restaurant 4 : {'name': 'NooRi', 'rating': '4,8', 'price': '10–20 €', 'restaurant_type': 'Restaurant de cuisine fusion asiati

In [17]:
#pour paris 11eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+11/@48.8756901,2.3169646,7972m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_11eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Rey - Restaurant Paris 11', 'rating': '3,9', 'price': '20–30 €', 'restaurant_type': 'Restaurant français', 'description': 'Plats classiques de brasserie, comme le steak-frites et les assiettes de charcuterie, dans une salle confortable aux murs en lambris.', 'address': '130 Rue de la Roquette, 75011 Paris'}
Restaurant 2 : {'name': 'Qui plume la lune', 'rating': '4,7', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant français', 'description': 'Menus fixes de mets français innovants accompagnés de vin, proposés dans un restaurant haut de gamme élégant.', 'address': '50 Rue Amelot, 75011 Paris'}
Restaurant 3 : {'name': 'Les Petits Papiers', 'rating': '4,6', 'price': '30–40 €', 'restaurant_type': 'Restaurant français', 'description': 'Ce petit bistrot moderne orné de tons rouges propose une cuisine fine dans une atmosphère chaleureuse.', 'address': '18 Rue Oberkampf, 75011 Paris'}
Restaurant 4 : {'name': 'Pierre Sang',

In [18]:
#pour paris 12eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+12/@48.8756901,2.3169646,7972m/data=!3m1!1e3?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_12eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': "L'Aubergeade", 'rating': '4,8', 'price': '30–40 €', 'restaurant_type': 'Restaurant', 'description': 'Brasserie typique et cosy proposant des plats français traditionnels comme le steak tartare et des plats de poissons/fruits de mer.', 'address': '17 Rue Chaligny, 75012 Paris'}
Restaurant 2 : {'name': 'Virtus', 'rating': '4,8', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant gastronomique', 'description': 'Restaurant branché au décor vintage proposant des menus dégustation innovants et raffinés, et du vin.', 'address': '29 Rue de Cotte, 75012 Paris'}
Restaurant 3 : {'name': "L'Alchimiste", 'rating': '4,7', 'price': '20–30 €', 'restaurant_type': 'Restaurant français', 'description': 'Cuisine française créative dans une ambiance bistro, avec luminaires Art nouveau et grandes ardoises murales.', 'address': '181 Rue de Charenton, 75012 Paris'}
Restaurant 4 : {'name': "L'Ébauchoir", 'rating': '4,7', 'price': '€€', 'restaur

In [19]:
#pour paris 13eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+13/@48.8573678,2.2731491,7975m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_13eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'SOURIRE le restaurant', 'rating': '4,7', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant gastronomique', 'description': 'Ce restaurant sert une cuisine française raffinée dans une salle chic et moderne rehaussée de velours bleu.', 'address': '15 Rue de la Santé, 75013 Paris'}
Restaurant 2 : {'name': 'Under The Sea Restaurant - Ephemera', 'rating': '4,7', 'price': '20–40 €', 'restaurant_type': 'Restaurant', 'description': 'Spectaculaire restaurant branché inspiré des fonds marins, servant des plats de poissons/fruits de mer du monde et des cocktails.', 'address': '160 Av. de France, 75013 Paris'}
Restaurant 3 : {'name': 'Chez Gladines Butte aux Cailles', 'rating': '4,4', 'price': '20–30 €', 'restaurant_type': 'Restaurant basque', 'description': 'Plats mijotés, escalopes et autres mets rustiques originaires du sud-ouest de la France, proposés dans une salle à manger cosy.', 'address': '30 Rue des cinq Diamants, 75013 P

In [20]:
#pour paris 14eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+14/@48.8329671,2.3124079,1995m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_14eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Jéroboam', 'rating': '4,8', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': 'Cuisine créative de bistrot français aux influences du monde entier dans une brasserie pittoresque à la décoration colorée.', 'address': '72 Rue Didot, 75014 Paris'}
Restaurant 2 : {'name': 'Chez Papa', 'rating': '4,3', 'price': '20–30 €', 'restaurant_type': 'Restaurant français', 'description': "Spécialités traditionnelles du Sud-Ouest dans une atmosphère populaire, autour de tables en bois d'auberge.", 'address': '6 Rue Gassendi, 75014 Paris'}
Restaurant 3 : {'name': 'Restaurant 14 Juillet', 'rating': '4,5', 'price': '20–30 €', 'restaurant_type': 'Restaurant français', 'description': "Restaurant cosy proposant des terrines, des profiteroles et d'autres classiques accompagnés d'un large choix de vins.", 'address': '99 Rue Didot, 75014 Paris'}
Restaurant 4 : {'name': 'L’Essentiel - Restaurant Paris 14', 'rating': '4,3', 

In [21]:
#pour paris 15eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+15/@48.8330299,2.2815079,7979m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_15eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': '750g la table I Porte de Versailles', 'rating': '4,6', 'price': '20–30 €', 'restaurant_type': 'Restaurant français', 'description': "Brasserie animée servant des plats et des vins français. Les portes en accordéon s'ouvrent pour laisser entrer l'extérieur.", 'address': '397 Rue de Vaugirard, 75015 Paris'}
Restaurant 2 : {'name': 'Le Relais du 15ème', 'rating': '4,7', 'price': '20–30 €', 'restaurant_type': 'Restaurant italien', 'description': 'NaN', 'address': '78 Av. Emile Zola, 75015 Paris'}
Restaurant 3 : {'name': "L'Antre Amis", 'rating': '4,6', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': 'Tapas raffinées, menus fixes et large choix de vins proposés dans un restaurant élégant doté de tables en extérieur.', 'address': '9 Rue Bouchut, 75015 Paris'}
Restaurant 4 : {'name': "L'Antre Amis", 'rating': '4,6', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': 'Tapas raffinées, m

In [22]:
#pour paris 16eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+16/@48.8331282,2.2815079,7979m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_16eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Restaurant Étude Paris 16', 'rating': '4,6', 'price': '+ de 100 €', 'restaurant_type': 'Restaurant', 'description': "Restaurant gastronomique à l'approvisionnement responsable proposant des plats français contemporains et végans pour certains.", 'address': '14 Rue du Bouquet de Longchamp, 75116 Paris'}
Restaurant 2 : {'name': 'Le Bois', 'rating': '4,3', 'price': '20–40 €', 'restaurant_type': 'Restaurant français', 'description': 'Adaptée aux familles, cette brasserie branchée avec une terrasse est prisée pour sa salade César, ses burgers et sa crème caramel.', 'address': '29 Rue Bois le Vent, 75016 Paris'}
Restaurant 3 : {'name': 'Le Paris Seize', 'rating': '4,6', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': "Dans un cadre décontracté et familial, restaurant à l'ambiance de bistrot italien pour des plats typiques.", 'address': '18 Rue des Belles Feuilles, 75116 Paris'}
Restaurant 4 : {'name': 'XVIe Avenue

In [23]:
#pour paris 17eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+17/@48.8806927,2.2874628,1993m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_17eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Paris 17', 'rating': '4,7', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': "Plats français traditionnels dans une brasserie cosy au charme d'antan décorée d'affiches vintage.", 'address': '41 Rue Guersant, 75017 Paris'}
Restaurant 2 : {'name': 'Le Paris 17', 'rating': '4,7', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': "Plats français traditionnels dans une brasserie cosy au charme d'antan décorée d'affiches vintage.", 'address': '41 Rue Guersant, 75017 Paris'}
Restaurant 3 : {'name': 'Les Demoiselles Restaurant Paris 17', 'rating': '4,5', 'price': '€€', 'restaurant_type': 'Restaurant français', 'description': "Dotée d'une terrasse, cette brasserie de style années 1950 à l'ambiance rétro sert des plats français toute la journée.", 'address': '23 Rue Léon Jost, 75017 Paris'}
Restaurant 4 : {'name': 'Restaurant La Table du Caviste bio Paris 17', 'rating': '4,7', 'price':

In [24]:
#pour paris 18eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+18/@48.8807555,2.2565628,7971m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_18eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': 'Le Relais Gascon montmartre paris18e', 'rating': '4,3', 'price': '20–30 €', 'restaurant_type': 'Restaurant français', 'description': "Restaurant traditionnel doté d'une terrasse, servant des plats maison du Sud-Ouest de la France.", 'address': '13 Rue Joseph de Maistre, 75018 Paris'}
Restaurant 2 : {'name': 'Au Clair de la Lune', 'rating': '4,2', 'price': '20–30 €', 'restaurant_type': 'Restaurant', 'description': "Restaurant pittoresque et original avec terrasse, servant des plats de brasserie tels que soupe à l'oignon et magret de canard.", 'address': '9 Rue Poulbot, 75018 Paris'}
Restaurant 3 : {'name': 'Chez Plumeau', 'rating': '4,1', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': "Plats traditionnels comme escargots, soupe à l'oignon et grillades de viande servis sur une terrasse verdoyante ou dans des salles à manger cosy.", 'address': '4 Pl. du Calvaire, 75018 Paris'}
Restaurant 4 : {'name': 'Au Clair

In [25]:
#pour paris 19eme arrondissement:
link="https://www.google.fr/maps/search/restaurant+paris+19/@48.8890345,2.2902335,7970m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_19eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': '19 darù', 'rating': '4,0', 'price': '30–40 €', 'restaurant_type': 'Restaurant italien', 'description': 'Restaurant intime, décorée de toiles vives par la propriétaire, qui propose des plats italiens traditionnels.', 'address': '19 Rue Daru, 75008 Paris'}
Restaurant 2 : {'name': 'Trois B', 'rating': '4,7', 'price': 'NaN', 'restaurant_type': 'Restaurant', 'description': "Bistrot à l'ambiance détendue servant des plats du jour, vins bio et bières locales dans une salle chaleureuse et lumineuse.", 'address': '48 Av. Jean Jaurès, 75019 Paris'}
Restaurant 3 : {'name': 'Mensae', 'rating': '4,5', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': 'Restaurant chic avec des spécialités françaises raffinées, des assiettes à partager et du vin dans un cadre moderne et minimaliste.', 'address': '23 Rue Melingue, 75019 Paris'}
Restaurant 4 : {'name': 'SYMPHONY | Restaurant Italien | Restaurant Villette | Restaurant Paris 19'

In [26]:
#pour paris 20eme arrondssement:
link="https://www.google.fr/maps/search/restaurant+paris+19/@48.8890345,2.2902335,7970m/data=!3m2!1e3!4b1?entry=ttu&g_ep=EgoyMDI1MDEwOC4wIKXMDSoASAFQAw%3D%3D"
output_file="data_in_progress/restaurants_paris_20eme.csv"
scrape_google_maps(link, n, output_file)

Élément de scroll trouvé avec succès !
Restaurant 1 : {'name': '19 darù', 'rating': '4,0', 'price': '30–40 €', 'restaurant_type': 'Restaurant italien', 'description': 'Restaurant intime, décorée de toiles vives par la propriétaire, qui propose des plats italiens traditionnels.', 'address': '19 Rue Daru, 75008 Paris'}
Restaurant 2 : {'name': 'Trois B', 'rating': '4,7', 'price': 'NaN', 'restaurant_type': 'Restaurant', 'description': "Bistrot à l'ambiance détendue servant des plats du jour, vins bio et bières locales dans une salle chaleureuse et lumineuse.", 'address': '48 Av. Jean Jaurès, 75019 Paris'}
Restaurant 3 : {'name': 'Mensae', 'rating': '4,5', 'price': '€€', 'restaurant_type': 'Restaurant', 'description': 'Restaurant chic avec des spécialités françaises raffinées, des assiettes à partager et du vin dans un cadre moderne et minimaliste.', 'address': '23 Rue Melingue, 75019 Paris'}
Restaurant 4 : {'name': 'SYMPHONY | Restaurant Italien | Restaurant Villette | Restaurant Paris 19'

In [27]:
import pandas as pd
#on ouvre les fichiers csv et on les concatene en un seul
#on enleve les doublons
#on ouvre tous les fichiers qui commencent par restaurants_paris
import glob
files = glob.glob("data_in_progress/restaurants_paris*.csv")
dataframes = []
for file in files:
    df = pd.read_csv(file)
    dataframes.append(df)
#concatenation
df = pd.concat(dataframes)
#enlever les doublons
df = df.drop_duplicates()
#sauvegarde
df.to_csv("restaurants_paris_final.csv", index=False)

In [29]:
df.shape

(334, 6)

: 

In [28]:
#on affiche les differents types de restaurants
print(df["restaurant_type"].value_counts())

restaurant_type
Restaurant français                                    143
Restaurant                                              85
Restaurant gastronomique                                24
Restaurant italien                                      20
Brasserie                                                9
Bistro                                                   9
Restaurant libanais                                      6
Cuisine française moderne                                3
Restaurant vietnamien                                    3
Restaurant chinois                                       3
Restaurant de cuisine fusion asiatique                   2
Restaurant de poisson                                    2
Restaurant indien                                        2
Restaurant asiatique                                     2
Restaurant méditerranéen                                 2
Restaurant de spécialités alsaciennes                    1
Bar à tapas                             